# Pulsed Laser Deposition Systems
Notebook with specific calculations that are useful for pulsed laser deposition experiments with University of Leeds systems. See Calculations.ipynb for typical laser calculations.

* B45 System
* B11 System

In [1]:
from numpy import pi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Laser Beam Area and Fluence

In [2]:
# Helper functions
def calc_F(E, area):
    """Convert pulse energy [J] and beam area [cm^2] to average fluence [J/cm2]"""
    return E / area

def calc_E(F, area):
    """Convert average fluence [J/cm2] and beam area [cm^2] to pulse energy [uJ]"""
    return F * area * 1E6

In [3]:
# Theoretical calculation of beam area in b45
aoi = 45  # Angle of incidence of laser on target [degrees]
minor = 27 # Minor axis of ellipse [um], Jaya calculated based on Gaussian optics and assuming a low divergence.
major = minor/np.sin(np.deg2rad(90-aoi)) # Major axis of ellipse [um]
area_b45 = pi * major * minor * 1E-8 # Area of the ellipse on the target [cm^2]
print('B45: Ellipse major axis = {:.2f} um, minor axis = {:.2f} um and area = {:.3g} cm2'.format(minor, major, area_b45))

B45: Ellipse major axis = 27.00 um, minor axis = 38.18 um and area = 3.24e-05 cm2


In [4]:
# Theoretical calculation of beam area in b11 - use only if direct measurement not possible
aoi = 60  # Angle of incidence of laser on target [degrees]
minor = 75/2 # Minor axis of ellipse [um]
major = minor/np.sin(np.deg2rad(90-aoi)) # Major axis of ellipse [um]

area = pi * major * minor * 1E-8 # Area of the ellipse on the target [cm^2]
print('B11: Theoretical Ellipse major axis = {:g} um, minor axis = {:g} um and area = {:.3g} cm2'.format(minor, major, area))

B11: Theoretical Ellipse major axis = 37.5 um, minor axis = 75 um and area = 8.84e-05 cm2


In [5]:
## Experimental measurement of beam area - see Calculations.ipynb for measurement options
major = 70.53  # Elipse axis [um]
minor = 27.75  # Elipse axis [um]
area = pi * major * minor * 1E-8 # Area of the ellipse on the target [cm^2]
print('B11: Experimental Ellipse major axis = {:g} um, minor axis = {:g} um and area = {:.3g} cm2'.format(minor, major, area))

B11: Experimental Ellipse major axis = 27.75 um, minor axis = 70.53 um and area = 6.15e-05 cm2


In [6]:
# Ratio of beam areas b11/b45 - need to adjust energy in accordance if transitioning between systems
b45_to_b11 = area / area_b45
print('B11 to B45 beam area ratio {:.2f}'.format(b45_to_b11))

E_b45 = 65E-6
E = E_b45*b45_to_b11

print('B45: E = {:.2f} uJ and F_avg = {:.2f} J/cm2; Bll: E = {:.2f} uJ'.format(E_b45*1E6, calc_F(E_b45, area_b45), E*1E6))

B11 to B45 beam area ratio 1.90
B45: E = 65.00 uJ and F_avg = 2.01 J/cm2; Bll: E = 123.40 uJ


In [7]:
# Energies to set for logarithmically spaced Fluences
Fmin = 0.7  # Minimum fluence [J/cm2]
Fmax = 4.0  # Maximum fluence [J/cm2]
num = 5  # Number of steps

df = pd.DataFrame({'F_avg':np.geomspace(Fmin, Fmax, num=num)})
df['E_applied'] = calc_E(df['F_avg'], area)
df = df.sort_values(['E_applied', 'F_avg'])
df = df.round({'F_avg': 2, 'E_applied': 1})
df.to_clipboard()
df

,F_avg,E_applied
0,0.70,43.0
1,1.08,66.5
2,1.67,102.9
3,2.59,159.1
4,4.00,245.9


In [26]:
# Pulser per spot/area
v = 10  # Velocity of raster [mm/s]
f = 1E3  # Rep rate of laser [Hz]
D = 75E-3  # Diameter of spot in rastering axis [mm]

# Average pulses per area = length drawn per sec / rastering speed
print('Average number of pulses per unit area {:.2f}'.format(f*D / v))

Average number of pulses per unit area 7.50


# Atmosphere
Vary pressure in steps of increasing order of magnitude

In [22]:
# Chamber pressure conversions
mbarr_to_mtorr = 750
mtorr_to_mbarr = 1/750

pmin = 1E-2  # Minimum pressure [mbarr]
pmax = 1  # Maximum pressure [mbarr]
num = 3  # Number of steps

# Create a table of pressures required
p = np.geomspace(pmin, pmax, num=num)
p = np.insert(p, 0, 0)
df = pd.DataFrame({'mbarr':p})
df['mtorr'] = df['mbarr'] * mbarr_to_mtorr
df

,mbarr,mtorr
0,0.00,0.0
1,0.01,7.5
2,0.10,75.0
3,1.00,750.0
